In [1]:
import pandas as pd
import sqlite3

In [2]:
excel_file = 'Data-startupticker.xlsx'  
sheet1 = 'Companies'  
sheet2 = 'Deals'

# Read Excel sheets
df1 = pd.read_excel(excel_file, sheet_name=sheet1)
df2 = pd.read_excel(excel_file, sheet_name=sheet2)


# Df preprocessing
df1 = df1.drop_duplicates(subset="Title", keep="first")


# Connect to DB
conn = sqlite3.connect('my_database.db')
conn.execute("PRAGMA foreign_keys = ON;")
cursor = conn.cursor()

# Drop tables if they exist
cursor.execute("DROP TABLE IF EXISTS table2")
cursor.execute("DROP TABLE IF EXISTS table1")

# Create table1
cursor.execute('''
    CREATE TABLE table1 (
        Code TEXT PRIMARY KEY,
        Title TEXT UNIQUE,
        Industry TEXT,
        Vertical TEXT,
        Canton TEXT,
        "Spin-offs" TEXT,
        City TEXT,
        Year INTEGER,
        Highlights TEXT,
        "Gender CEO" TEXT,
        OOB BOOLEAN,
        Funded BOOLEAN,
        Comment TEXT
    )
''')

# Create table2 with foreign key to table1.Title
cursor.execute('''
    CREATE TABLE table2 (
        Id TEXT PRIMARY KEY,
        Investors TEXT,
        Amount REAL,
        Valuation REAL,
        Comment TEXT,
        URL TEXT,
        Confidential BOOLEAN,
        "Amount confidential" BOOLEAN,
        "Date of the funding round" TEXT,
        Type TEXT,
        Phase TEXT,
        Canton TEXT,
        Company TEXT,
        "Gender CEO" TEXT,
        FOREIGN KEY (Company) REFERENCES table1(Title)
    )
''')

# Insert data from DataFrames
df1.to_sql('table1', conn, if_exists='append', index=False)
df2.to_sql('table2', conn, if_exists='append', index=False)


3902

In [3]:
# Confirm
print(pd.read_sql("SELECT * FROM table1", conn).head(1))
print(pd.read_sql("SELECT * FROM table2", conn).head(1))

              Code     Title Industry Vertical Canton Spin-offs  City    Year  \
0  CHE-384.775.108  Noury AG     None     None   Bern      None  None  2021.0   

  Highlights Gender CEO  OOB  Funded Comment  
0       None       None    0       0    None  
      Id Investors  Amount  Valuation Comment  \
0  S4126   Elastic     NaN        NaN    None   

                                                 URL  Confidential  \
0  https://www.elastic.co/de/about/press/elastic-...             0   

   Amount confidential Date of the funding round  Type Phase Canton  \
0                    0       2021-10-14 00:00:00  EXIT  None     ZH   

             Company Gender CEO  
0  optimyze.cloud AG       Male  


In [4]:
conn.close()